In [137]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
df_train = pd.read_csv("train.csv")
df_train.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [138]:
print(df_train.shape)
#target variables
print(df_train["breed_category"].value_counts())
print(df_train["pet_category"].value_counts())

(18834, 11)
0.0    9000
1.0    8357
2.0    1477
Name: breed_category, dtype: int64
2    10621
1     7184
4      941
0       88
Name: pet_category, dtype: int64


By looking at above obsevations dataset is not imabalnced or completely balanced. Because in breed 0,1 classes are having almost same no.of datapoints. 2.0 class having very less datapoints. Same thing with pet as well.

In [139]:
print(df_train.isnull().sum())
print(df_train[df_train["breed_category"]==2.0].condition.isnull().sum())
print(df_train[df_train["breed_category"]==1.0].condition.isnull().sum())
print(df_train[df_train["breed_category"]==0.0].condition.isnull().sum())

pet_id               0
issue_date           0
listing_date         0
condition         1477
color_type           0
length(m)            0
height(cm)           0
X1                   0
X2                   0
breed_category       0
pet_category         0
dtype: int64
1477
0
0


Here you can observe that all null values are related to breed_category=2.0 class label.

In [140]:
df_train["condition"].fillna(3,inplace = True)
print(df_train.shape)
df_train.head()

(18834, 11)


,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,3.0,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [ ]:
# del df_train["pet_category"]
# df_train.head(2)

In [102]:
# calculate duplicates
dups = df_train.duplicated()
# report if there are any duplicates
print(dups.any())
# list all duplicate rows
print(df_train[dups])
# delete duplicate rows
# df.drop_duplicates(inplace=True)

False
Empty DataFrame
Columns: [pet_id, issue_date, listing_date, condition, color_type, length(m), height(cm), X1, X2, breed_category, pet_category]
Index: []


In [141]:
#features: condition is categorical variable
print(df_train["color_type"].value_counts())


Black                4620
White                2453
Brown                1791
Brown Tabby          1687
Tan                  1349
Blue                  852
Orange Tabby          791
Red                   526
Brown Brindle         496
Tricolor              469
Blue Tabby            386
Tortie                366
Calico                343
Gray                  307
Chocolate             259
Torbie                242
Cream Tabby           191
Sable                 167
Cream                 162
Fawn                  159
Yellow                143
Buff                  125
Lynx Point            117
Blue Merle            104
Seal Point             78
Black Brindle          66
Gray Tabby             65
Black Tabby            55
Flame Point            52
Orange                 39
Brown Merle            39
Black Smoke            32
Gold                   31
Tortie Point           26
Silver                 24
Red Tick               23
Blue Tick              21
Blue Point             20
Lilac Point 

In [142]:
from sklearn.preprocessing import OneHotEncoder# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(df_train[['color_type']]).toarray())# merge with main df train_df on key values
df_train = df_train.join(enc_df)
del df_train["color_type"]

df_train.head(2)

,pet_id,issue_date,listing_date,condition,length(m),height(cm),X1,X2,breed_category,pet_category,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,0.80,7.78,13,9,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,0.72,14.19,13,9,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [143]:
df_train['issue_date'] = pd.to_datetime(df_train.issue_date)
df_train['listing_date'] = pd.to_datetime(df_train.listing_date)
df_train['issue_date_year'] = df_train.issue_date.dt.year
df_train["listing_date_year"] = df_train.listing_date.dt.year
df_train["issue_date_month"] = df_train.issue_date.dt.month
df_train["listing_date_month"] = df_train.listing_date.dt.month
del df_train["issue_date"]
del df_train["listing_date"]
df_train.head(2)

,pet_id,condition,length(m),height(cm),X1,X2,breed_category,pet_category,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,issue_date_year,listing_date_year,issue_date_month,listing_date_month
0,ANSL_69903,2.0,0.80,7.78,13,9,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016,2016,7,9
1,ANSL_66892,1.0,0.72,14.19,13,9,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2013,2018,11,12


In [144]:
df_train.shape

(18834, 68)

In [107]:
from sklearn.model_selection import train_test_split
# split data into X and y
X = df_train.drop(['breed_category','pet_category','pet_id','height(cm)'], axis=1) # here droping value column wise i,e why axis=1
y_breed = df_train['breed_category'].values
y_pet = df_train['pet_category'].values

# split train data into train and cv sets
seed = 7
test_size = 0.33
X_train_breed, X_cv_breed, y_train_breed, y_cv_breed = train_test_split(X, y_breed, test_size=test_size, random_state=seed)
X_train_pet, X_cv_pet, y_train_pet, y_cv_pet = train_test_split(X, y_pet, test_size=test_size, random_state=seed)
print(X_train_breed.shape,X_train_pet.shape)
print(X_cv_breed.shape,X_cv_pet.shape)
print(y_train_breed.shape, y_train_pet.shape)
print(y_cv_breed.shape, y_cv_pet.shape)


(12618, 64) (12618, 64)
(6216, 64) (6216, 64)
(12618,) (12618,)
(6216,) (6216,)


In [108]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score

# fit model on breed training data
model1 = XGBClassifier()
model1.fit(X_train_breed, y_train_breed)
# make predictions for cv data of breed traget variable
y_pred_breed = model1.predict(X_cv_breed)
predictions_breed = [round(value) for value in y_pred_breed]
# evaluate predictions
accuracy = accuracy_score(y_cv_breed, predictions_breed)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# fit model on breed training data
model2 = XGBClassifier()
model2.fit(X_train_pet, y_train_pet)
# make predictions for cv data of pet traget variable
y_pred_pet = model2.predict(X_cv_pet)
predictions_pet = [round(value) for value in y_pred_pet]
# evaluate predictions
accuracy = accuracy_score(y_cv_pet, predictions_pet)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.09%
Accuracy: 87.68%


In [109]:
from sklearn.metrics import f1_score
s1 = f1_score(y_cv_breed,predictions_breed,average='weighted')
s2 = f1_score(y_cv_pet,predictions_pet,average='weighted')
print(s1,s2)
sc = (s1+s2)/2
score = sc*100
print(score)

0.900579873340035 0.8733195037321803
88.69496885361076


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
# First create the base model to tune
xgb_model = GradientBoostingClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 50 different combinations, and use all available cores
xgb_random = RandomizedSearchCV(estimator = xgb_model, param_distributions = random_grid, cv = 3, verbose=2, random_state=42, n_jobs = -1)
xgb_random.fit(X_train_breed, y_train_breed)

xgb_random.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.4min finished


{'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 55}

In [ ]:
#Training model with best parameters which we got from RandomsearchCV
from sklearn.ensemble import GradientBoostingClassifier
model3 = GradientBoostingClassifier(max_depth = 27, max_features = 'sqrt', min_samples_leaf = 2,min_samples_split=10,n_estimators=50)

model3.fit(X_train_breed, y_train_breed)

y_pred_breed = model3.predict(X_cv_breed)
predictions_breed = [round(value,5) for value in y_pred_breed]

# evaluate predictions
accuracy = accuracy_score(y_cv_breed, predictions_breed)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.01%


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
# First create the base model to tune
xgb_model = GradientBoostingClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 50 different combinations, and use all available cores
xgb_random_pet = RandomizedSearchCV(estimator = xgb_model, param_distributions = random_grid, cv = 3, verbose=2, random_state=42, n_jobs = -1)
xgb_random_pet.fit(X_train_pet, y_train_pet)

xgb_random_pet.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  7.3min finished


{'max_depth': 27,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'n_estimators': 50}

In [ ]:
#Training model with best parameters which we got from RandomsearchCV
from sklearn.ensemble import GradientBoostingClassifier
model4 = GradientBoostingClassifier(max_depth = 27, max_features = 'sqrt', min_samples_leaf = 2,min_samples_split=10,n_estimators=50)

model4.fit(X_train_pet, y_train_pet)

y_pred_pet = model4.predict(X_cv_pet)
predictions_pet = [round(value,5) for value in y_pred_pet]

# evaluate predictions
accuracy = accuracy_score(y_cv_pet, predictions_pet)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.27%


In [ ]:
from sklearn.metrics import f1_score
#here i am taking model1 for breed and model4 for pet trained models 
s1 = f1_score(y_cv_breed,predictions_breed,average='weighted')
s2 = f1_score(y_cv_pet,predictions_pet,average='weighted')
print(s1,s2)
sc = (s1+s2)/2
score = sc*100
print(score)

0.9000086251790249 0.9008797666678139
90.04441959234194


In [148]:
df_test = pd.read_csv("test.csv")
df_test.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [149]:
enc_df = pd.DataFrame(enc.transform(df_test[['color_type']]).toarray())# merge with main df train_df on key values
df_test = df_test.join(enc_df)
del df_test["color_type"]

df_test.head(2)

,pet_id,issue_date,listing_date,condition,length(m),height(cm),X1,X2,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,0.87,42.73,0,7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,0.06,6.71,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [150]:
df_test['issue_date'] = pd.to_datetime(df_test.issue_date)
df_test['listing_date'] = pd.to_datetime(df_test.listing_date)
df_test['issue_date_year'] = df_test.issue_date.dt.year
df_test["listing_date_year"] = df_test.listing_date.dt.year
df_test["issue_date_month"] = df_test.issue_date.dt.month
df_test["listing_date_month"] = df_test.listing_date.dt.month
del df_test["issue_date"]
del df_test["listing_date"]
df_test.head(2)

,pet_id,condition,length(m),height(cm),X1,X2,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,issue_date_year,listing_date_year,issue_date_month,listing_date_month
0,ANSL_75005,0.0,0.87,42.73,0,7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2005,2017,8,9
1,ANSL_76663,1.0,0.06,6.71,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018,2019,11,5


In [151]:
print(df_test.isnull().sum())
df_test["condition"].fillna(3,inplace = True)

pet_id                  0
condition             619
length(m)               0
height(cm)              0
X1                      0
                     ... 
55                      0
issue_date_year         0
listing_date_year       0
issue_date_month        0
listing_date_month      0
Length: 66, dtype: int64


In [116]:
X_test = df_test.drop(['pet_id','height(cm)'], axis=1)

In [ ]:

y_pred_breed = model1.predict(X_test)
#storing predicted values in list
predictions_breed = [round(value,5) for value in y_pred_breed]



In [ ]:
y_pred_pet = model2.predict(X_test)
#storing predicted values in list
predictions_pet = [round(value,5) for value in y_pred_pet]

In [ ]:
df_test = pd.read_csv("test.csv")
final_test_df = df_test.drop(["issue_date","listing_date","condition","color_type","length(m)","height(cm)","X1","X2"],axis=1)
final_test_df["breed_category"] = predictions_breed
final_test_df["pet_category"] = predictions_pet
final_test_df.head()


,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,2
4,ANSL_72871,0.0,2


In [ ]:
final_test_df.to_csv("Petadp_Output.csv",index=False)

In [ ]:
pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19877 sha256=516f3a3bcc38f9b9e20058f04ba95327e16a6362f65ec3f85b31c113c88bf9f6
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [ ]:
model5 = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                      n_estimators=100, max_depth=3)
    
model5 = model5.fit(S_train_breed, y_train_breed)
y_pred_breed = model5.predict(S_cv_breed)
print('Final prediction score: [%.8f]' % accuracy_score(y_cv_breed, y_pred_breed))

Final prediction score: [0.90041828]


In [ ]:
model6 = XGBClassifier()
    
model6 = model6.fit(S_train_pet, y_train_pet)
y_pred_pet = model6.predict(S_cv_pet)
print('Final prediction score: [%.8f]' % accuracy_score(y_cv_pet, y_pred_pet))

Final prediction score: [0.87628700]


In [ ]:
y_pred_breed = model3.predict(X_test)
#storing predicted values in list
predictions_breed = [round(value,5) for value in y_pred_breed]

In [ ]:
y_pred_pet = model4.predict(X_test)
#storing predicted values in list
predictions_pet = [round(value,5) for value in y_pred_pet]

In [ ]:
df_test = pd.read_csv("test.csv")
final_test_df = df_test.drop(["issue_date","listing_date","condition","color_type","length(m)","height(cm)","X1","X2"],axis=1)
final_test_df["breed_category"] = predictions_breed
final_test_df["pet_category"] = predictions_pet
final_test_df.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,2
4,ANSL_72871,0.0,2


In [ ]:
final_test_df.to_csv("Petadp_Output2.csv",index=False)

In [110]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

estimators = [
     ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
     ('svr', make_pipeline(StandardScaler(),
                           LinearSVC(random_state=42)))
 ]
model5 = StackingClassifier(
     estimators=estimators, final_estimator=LogisticRegression()
)

model5.fit(X_train_breed, y_train_breed).score(X_cv_breed, y_cv_breed)

0.8935006435006435

In [117]:
y_pred_breed = model5.predict(X_test)
#storing predicted values in list
predictions_breed = [round(value,5) for value in y_pred_breed]

In [118]:
estimators = [
     ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
     ('svr', make_pipeline(StandardScaler(),
                           LinearSVC(random_state=42)))
 ]
model6 = StackingClassifier(
     estimators=estimators, final_estimator=LogisticRegression()
)

model6.fit(X_train_pet, y_train_pet).score(X_cv_pet, y_cv_pet)

0.8941441441441441

In [119]:
y_pred_pet = model6.predict(X_test)
#storing predicted values in list
predictions_pet = [round(value,5) for value in y_pred_pet]

In [120]:
df_test = pd.read_csv("test.csv")
final_test_df = df_test.drop(["issue_date","listing_date","condition","color_type","length(m)","height(cm)","X1","X2"],axis=1)
final_test_df["breed_category"] = predictions_breed
final_test_df["pet_category"] = predictions_pet
final_test_df.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,1
4,ANSL_72871,0.0,2


In [121]:
final_test_df.to_csv("Petadp_Output5.csv",index=False)

In [128]:
from sklearn.ensemble import GradientBoostingClassifier
estimators = [
     ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
     ('svr', make_pipeline(StandardScaler(),
                           GradientBoostingClassifier(random_state=42)))
 ]
model7 = StackingClassifier(
     estimators=estimators, final_estimator=LogisticRegression())

model7.fit(X_train_breed, y_train_breed).score(X_cv_breed, y_cv_breed)

0.8986486486486487

In [129]:
y_pred_breed = model7.predict(X_test)
#storing predicted values in list
predictions_breed = [round(value,5) for value in y_pred_breed]

In [130]:
estimators = [
     ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
     ('svr', make_pipeline(StandardScaler(),
                           GradientBoostingClassifier(random_state=42)))
 ]
model8 = StackingClassifier(
     estimators=estimators, final_estimator=LogisticRegression()
)

model8.fit(X_train_pet, y_train_pet).score(X_cv_pet, y_cv_pet)

0.8981660231660231

In [131]:
y_pred_pet = model8.predict(X_test)
#storing predicted values in list
predictions_pet = [round(value,5) for value in y_pred_pet]

In [132]:
df_test = pd.read_csv("test.csv")
final_test_df = df_test.drop(["issue_date","listing_date","condition","color_type","length(m)","height(cm)","X1","X2"],axis=1)
final_test_df["breed_category"] = predictions_breed
final_test_df["pet_category"] = predictions_pet
final_test_df.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,2
4,ANSL_72871,0.0,2


In [133]:
final_test_df.to_csv("Petadp_Output6.csv",index=False)

In [134]:
estimators = [
     ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
     ('svr', make_pipeline(StandardScaler(),
                           LinearSVC(random_state=42)))
 ]
model9 = StackingClassifier(
     estimators=estimators, final_estimator=LogisticRegression()
)

model9.fit(X_train_pet, y_train_pet).score(X_cv_pet, y_cv_pet)

y_pred_pet = model8.predict(X_test)
#storing predicted values in list
predictions_pet = [round(value,5) for value in y_pred_pet]

In [152]:
df_test["pet_category"]=predictions_pet
df_test.head(2)

,pet_id,condition,length(m),height(cm),X1,X2,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,issue_date_year,listing_date_year,issue_date_month,listing_date_month,pet_category
0,ANSL_75005,0.0,0.87,42.73,0,7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2005,2017,8,9,2
1,ANSL_76663,1.0,0.06,6.71,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018,2019,11,5,1


In [145]:
from sklearn.model_selection import train_test_split
# split data into X and y
X = df_train.drop(['breed_category','pet_id','height(cm)'], axis=1) # here droping value column wise i,e why axis=1
y_breed = df_train['breed_category'].values


# split train data into train and cv sets
seed = 7
test_size = 0.33
X_train_breed, X_cv_breed, y_train_breed, y_cv_breed = train_test_split(X, y_breed, test_size=test_size, random_state=seed)

print(X_train_breed.shape)
print(X_cv_breed.shape)
print(y_train_breed.shape)
print(y_cv_breed.shape)


(12618, 65)
(6216, 65)
(12618,)
(6216,)


In [146]:
from sklearn.ensemble import GradientBoostingClassifier
estimators = [
     ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
     ('svr', make_pipeline(StandardScaler(),
                           GradientBoostingClassifier(random_state=42)))
 ]
model10 = StackingClassifier(
     estimators=estimators, final_estimator=LogisticRegression())

model10.fit(X_train_breed, y_train_breed).score(X_cv_breed, y_cv_breed)

0.8984877734877735

In [ ]:
y_pred_breed = model10.predict(X_test)
#storing predicted values in list
predictions_breed = [round(value,5) for value in y_pred_breed]

In [153]:
X_test = df_test.drop(['pet_id','height(cm)'], axis=1)
y_pred_breed = model10.predict(X_test)
#storing predicted values in list
predictions_breed = [round(value,5) for value in y_pred_breed]

In [154]:
df_test = pd.read_csv("test.csv")
final_test_df = df_test.drop(["issue_date","listing_date","condition","color_type","length(m)","height(cm)","X1","X2"],axis=1)
final_test_df["breed_category"] = predictions_breed
final_test_df["pet_category"] = predictions_pet
final_test_df.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,2
4,ANSL_72871,0.0,2


In [155]:
final_test_df.to_csv("Petadp_Output7.csv",index=False)